In [1]:
import sys

from pathlib import Path
import numpy as np
import pandas as pd
import scipy.signal
from scipy import signal
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import RobustScaler
from tqdm import tqdm
import copy
import pickle
import lightgbm as lgb


import warnings
warnings.simplefilter("ignore")

In [2]:
class CFG:
    exp_num = 7
    n_folds = 10
    folds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    seed = 777
    local = True


In [3]:
DATA_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/ventilator-pressure-prediction")
OUTPUT_DIR = Path('./output/')
OOF_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_oofs_stacking")
SUB_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_subs_stacking")
PICKLE_DIR = Path("/home/knikaido/work/Ventilator-Pressure-Prediction/data/team_stacking_pickle")

In [4]:
sys.path.append('../../src/')
import utils as utils
from utils import Timer

In [5]:
train = pd.read_csv(DATA_DIR / "train.csv")
test = pd.read_csv(DATA_DIR / "test.csv")

In [6]:
oof_paths = sorted(list(OOF_DIR.rglob('*.npy'))+list(OOF_DIR.rglob('*.csv')))
sub_paths = sorted(list(SUB_DIR.rglob('*.npy'))+list(SUB_DIR.rglob('*.csv')))

len(oof_paths), len(sub_paths)

(32, 32)

In [7]:
with open(PICKLE_DIR / "all_stacking1_oof.npy", mode="rb") as f:
    oofs = np.load(f)

In [9]:
oofs = pd.DataFrame(oofs)
oofs.columns = [f"pred_{i}" for i in range(len(oofs.columns))]
oofs

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9
0,6.051358,6.008679,5.839750,5.916682,5.907529,5.523668,5.810369,5.970937,5.683621,5.805068
1,5.901721,5.824164,5.896797,5.886481,5.899961,5.929617,5.871547,5.902353,5.919608,5.846550
2,8.389902,8.267651,8.516665,8.561444,8.385892,8.095352,8.126698,8.220336,8.197915,7.975787
3,12.343451,12.020198,12.398529,12.385787,12.373862,12.134422,12.084424,12.261753,12.206296,12.002528
4,12.681964,12.505707,12.786497,12.760039,12.763080,12.600920,12.546120,12.702161,12.597127,12.502975
...,...,...,...,...,...,...,...,...,...,...
2290963,29.430931,29.476478,29.435886,29.438047,29.445435,29.459700,29.480509,29.465638,29.459790,29.423048
2290964,29.093206,29.113724,29.106075,29.106499,29.109507,29.138840,29.120060,29.103670,29.111308,29.086384
2290965,29.850384,29.908362,29.861372,29.865599,29.871944,29.883578,29.890086,29.886187,29.887238,29.874448
2290966,29.370970,29.400613,29.380976,29.381800,29.382881,29.397045,29.373083,29.392937,29.371835,29.382864


In [10]:
with open(PICKLE_DIR / "all_stacking1_sub.npy", mode="rb") as f:
    subs = np.load(f)

In [11]:
subs = pd.DataFrame(subs)
subs.columns = [f"pred_{i}" for i in range(len(subs.columns))]
subs

,pred_0,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6,pred_7,pred_8,pred_9
0,6.122457,6.234838,6.228402,6.223941,6.215096,6.270192,6.259692,6.246551,6.232782,6.239510
1,5.897243,5.966046,5.976529,5.962824,5.969975,5.970970,5.984572,5.972593,5.991468,5.973396
2,7.116149,7.154629,7.155302,7.151405,7.149951,7.154575,7.120526,7.143456,7.191395,7.121976
3,7.687282,7.767205,7.695719,7.696728,7.700007,7.730515,7.684567,7.731995,7.745194,7.688272
4,9.229881,9.256029,9.214653,9.217604,9.215394,9.231619,9.252678,9.217831,9.240966,9.231130
...,...,...,...,...,...,...,...,...,...,...
4023995,0.000000,0.000000,10.720507,11.235307,13.147164,13.164266,15.354063,14.852493,13.631219,13.676671
4023996,0.000000,0.000000,11.166886,11.625679,13.939608,13.322442,15.355318,14.720789,13.892657,14.014549
4023997,0.000000,0.000000,11.206486,11.646933,14.122984,13.530339,15.515092,15.015911,14.041680,14.213983
4023998,0.000000,0.000000,10.823473,12.016227,13.867440,13.157774,16.102294,15.561432,14.433167,14.409499


In [13]:
def get_raw_features(input_df, dataType = 'train'):
    colum = ['time_step', 'u_in', 'R', 'C']

    return input_df[colum]

In [14]:
def get_category_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    colum = ['R_C']
    rc_map = {'5_10': 0, '5_20': 1, '5_50': 2, '20_10': 3, '20_20': 4, '20_50': 5, '50_10': 6, '50_20': 7, '50_50': 8}
    
    output_df['R_C'] = [f'{r}_{c}' for r, c in zip(output_df['R'], output_df['C'])]
    output_df['R_C'] = output_df['R_C'].map(rc_map)

    return output_df[colum]

In [15]:
def get_simple_calc_features(input_df, dataType = 'train'):
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['time_delta'] = output_df.groupby('breath_id')['time_step'].diff().fillna(0)
    output_df['delta'] = output_df['time_delta'] * output_df['u_in']
    output_df['area'] = output_df.groupby('breath_id')['delta'].cumsum()

    output_df['cross']= output_df['u_in']*output_df['u_out']
    output_df['cross2']= output_df['time_step']*output_df['u_out']
    
    output_df['u_in_cumsum'] = (output_df['u_in']).groupby(output_df['breath_id']).cumsum()
    output_df['one'] = 1
    output_df['count'] = (output_df['one']).groupby(output_df['breath_id']).cumsum()
    output_df['u_in_cummean'] =output_df['u_in_cumsum'] / output_df['count']
    
    output_df['u_in_sqrt'] = output_df['u_in'].apply(lambda x: np.sqrt(x))
    output_df['u_in_sqrt_cumsum'] = output_df.groupby('breath_id')['u_in_sqrt'].cumsum()
    
    output_df = output_df.drop(['count','one'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [16]:
def get_agg_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    # Dict for aggregations
    create_feature_dict = {
        'u_in': [np.max, np.mean],
    }
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id']).agg(create_feature_dict)
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(start_time) + '_' + str(end_time))
            
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    
#     df_tmp = get_agg_window(start_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(start_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 1, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')
#     df_tmp = get_agg_window(end_time = 2, add_suffix = True).reset_index()
#     df_agg_feature = df_agg_feature.merge(df_tmp, how = 'left', on = 'breath_id')

    output_df = pd.merge(output_df, df_agg_feature, how='left', on='breath_id')
    
    output_df['u_in_diffmax'] = output_df['u_in_amax'] - output_df['u_in']
    output_df['u_in_diffmean'] = output_df['u_in_mean'] - output_df['u_in']
    
#     output_df = output_df.drop(['u_in_amax','u_in_mean'], axis=1)
    
    return output_df.iloc[:, c_num:]

In [17]:
def get_half_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['tmp'] = output_df['u_out']*(-1)+1 # inversion of u_out
    output_df['u_in_half'] = output_df['tmp'] * output_df['u_in']
    
#     u_in_half_max_dict = train.groupby('breath_id')['u_in_half'].max().to_dict()
#     train['u_in_half_max'] = train['breath_id'].map(u_in_half_max_dict)
#     u_in_half_min_dict = train.groupby('breath_id')['u_in_half'].min().to_dict()
#     train['u_in_half_min'] = train['breath_id'].map(u_in_half_min_dict)
    u_in_half_mean_dict = output_df.groupby('breath_id')['u_in_half'].mean().to_dict()
    output_df['u_in_half_mean'] = output_df['breath_id'].map(u_in_half_mean_dict)
#     u_in_half_std_dict = train.groupby('breath_id')['u_in_half'].std().to_dict()
#     train['u_in_half_std'] = train['breath_id'].map(u_in_half_std_dict)

    del output_df['u_in_half'], output_df['tmp']
    return output_df.iloc[:, c_num:]

In [18]:
def lowpass_filter(series, b, a):
    return signal.filtfilt(b, a, series)    

In [19]:
def get_filter_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    fp = 5 # 通過域端周波数[Hz]
    fs = 10 # 阻止域端周波数[Hz]
    gpass = 3 # 通過域端最大損失[dB]
    gstop = 40 # 阻止域端最小損失[dB]
    samplerate = 100

    fn = samplerate / 2   #ナイキスト周波数
    wp = fp / fn  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    
    def get_agg_window(start_time=0, end_time=3.0, add_suffix = False):
        
        df_tgt = output_df[(output_df['time_step'] >= start_time) & (output_df['time_step'] <= end_time)]
        df_feature = df_tgt.groupby(['breath_id'])['u_in'].apply(lowpass_filter, b=b, a=a)
        df_feature.name = 'u_in_filter'
                    
        return df_feature
    
    df_agg_feature = get_agg_window().reset_index()
    df_agg_feature = df_agg_feature.explode("u_in_filter").reset_index(drop=True)
    df_agg_feature['u_in_filter'] = df_agg_feature['u_in_filter'].astype(float)
        
    df_agg_feature['u_in_filter_cumsum'] = df_agg_feature.groupby('breath_id')['u_in_filter'].cumsum()

    return df_agg_feature.iloc[:, 1:]

In [20]:
def get_vib_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    output_df['u_out_diff'] = output_df['u_out'].diff()
    output_df['u_out_diff'].fillna(0, inplace=True)
    output_df['u_out_diff'].replace(-1, 0, inplace=True)
    uout1_df = output_df[output_df['u_out_diff']==1]
    
    first_df = output_df.loc[0::80,:]
    first_0_dict = dict(zip(first_df['id'], [0]*len(uout1_df)))

    output_df['u_in_diff'] = output_df['u_in'].diff()
    output_df['diff_sign'] = np.sign(output_df['u_in_diff'])
    output_df['sign_diff'] = output_df['diff_sign'].diff()
    output_df['tmp'] = output_df['id'].map(first_0_dict) # put 0, the 80row cycle
    output_df.iloc[0::80, output_df.columns.get_loc('sign_diff')] = output_df.iloc[0::80, output_df.columns.get_loc('tmp')]

    # Count the number of inversions, so take the absolute value and sum
    output_df['sign_diff'] = abs(output_df['sign_diff']) 
    sign_diff_dict = output_df.groupby('breath_id')['sign_diff'].sum().to_dict()
    output_df['diff_vib'] = output_df['breath_id'].map(sign_diff_dict)
    
    return output_df['sign_diff']

In [21]:
def add_time_features(out_df, input_df, dataType = 'train'):

    USE_LAG = [-2, -1, 1, 2, 3, 4]
    lag_map = {-2: 1, -1: 2, 1: 3, 2: 4, 3: 5, 4: 6}

    out_df['breath_id'] = input_df['breath_id']
    
    for lag in USE_LAG:
        out_df[f'breath_id_lag{lag_map[lag]}']=out_df['breath_id'].shift(lag).fillna(0)
        out_df[f'breath_id_lag{lag_map[lag]}same']=np.select([out_df[f'breath_id_lag{lag_map[lag]}']==out_df['breath_id']], [1], 0)

        # u_in_filter
        out_df[f'u_in_filter_lag_{lag_map[lag]}'] = out_df['u_in_filter'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_filter_diff_{lag_map[lag]}'] = out_df['u_in_filter'] - out_df[f'u_in_filter_lag_{lag_map[lag]}']
        # u_in_sqrt
        out_df[f'u_in_sqrt_lag_{lag_map[lag]}'] = out_df['u_in_sqrt'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_sqrt_diff_{lag_map[lag]}'] = out_df['u_in_sqrt'] - out_df[f'u_in_sqrt_lag_{lag_map[lag]}']

        # u_in 
        out_df[f'u_in_lag_{lag_map[lag]}'] = out_df['u_in'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']
        out_df[f'u_in_diff_{lag_map[lag]}'] = out_df['u_in'] - out_df[f'u_in_lag_{lag_map[lag]}']
        # u_out
        out_df[f'u_out_lag_{lag_map[lag]}'] = out_df['u_out'].shift(lag).fillna(0) * out_df[f'breath_id_lag{lag_map[lag]}same']

        # breath_time
    out_df[f'time_step_lag_{1}'] = out_df['time_step'].shift(1).fillna(0) * out_df[f'breath_id_lag{1}same']
    out_df[f'time_step_diff_{1}'] = out_df['time_step'] - out_df[f'time_step_lag_{1}']
        
    drop_columns = ['breath_id', 'time_step_lag_1']
    drop_columns += [f'breath_id_lag{lag_map[i]}' for i in USE_LAG]
    drop_columns += [f'breath_id_lag{lag_map[i]}same' for i in USE_LAG]
    out_df = out_df.drop(drop_columns, axis=1)
    out_df = out_df.fillna(0)
    
    return out_df

In [22]:
def get_oof_features(input_df, dataType = 'train'):
    
    output_df = copy.deepcopy(input_df)
    c_num = input_df.shape[1]
    
    for i in range(len(pred_cols)):
        output_df[f"pred_{i}"] = 0.
        output_df.loc[oof["u_out"] == 0, f"pred_{i}"] = _oof[f"oof{i}"].values
    

    
    return output_df['sign_diff']

In [23]:
def to_feature(input_df, dataType = 'train'):
    """input_df を特徴量行列に変換した新しいデータフレームを返す.
    """

    processors = [
        get_raw_features,
#         get_simple_calc_features,
#         get_agg_features,
#         get_vib_features,
#         get_half_features,
        get_category_features,
#         get_filter_features,
    ]

    out_df = pd.DataFrame()

    for func in tqdm(processors, total=len(processors)):
        with Timer(prefix='' + func.__name__ + ' '):
            _df = func(input_df, dataType)

        # 長さが等しいことをチェック (ずれている場合, func の実装がおかしい)
        assert len(_df) == len(input_df), func.__name__
        out_df = pd.concat([out_df, _df], axis=1)
#     out_df = utils.reduce_mem_usage(out_df)
#     out_df = add_time_features(out_df, input_df)
    out_df_cols = sorted(list(out_df))
    out_df = out_df[out_df_cols]
    
    return out_df

In [24]:
train_df = to_feature(train, dataType = 'train')
test_df = to_feature(test, dataType = 'test')

  0%|          | 0/2 [00:00<?, ?it/s]

get_raw_features  0.029[s]


  0%|          | 0/2 [00:00<?, ?it/s]

get_category_features  2.411[s]
get_raw_features  0.019[s]


100%|██████████| 2/2 [00:01<00:00,  1.26it/s]

get_category_features  1.524[s]


In [25]:
pred_cols = [f"pred_{i}" for i in range(len(oofs.columns))]

In [26]:
def add_oof_features(input_df, dataType = 'train'):
    
    if dataType == 'train':
        for i in range(len(pred_cols)):
            input_df[f"pred_{i}"] = 0.
            input_df.loc[train["u_out"] == 0, f"pred_{i}"] = oofs[f"pred_{i}"].values
        input_df['breath_id'] = train['breath_id']
        input_df = train_df.loc[train["u_out"] == 0].reset_index(drop=True)
    else:
        for i in range(len(pred_cols)):
            input_df[f"pred_{i}"] = 0.
            input_df.loc[:, f"pred_{i}"] = subs[f"pred_{i}"].values
        input_df['breath_id'] = test['breath_id']
        input_df = input_df.loc[test["u_out"] == 0].reset_index(drop=True)
      
    # v2
    input_df["pred_mean"] = np.mean(input_df[pred_cols].values, axis=1)
    input_df["pred_median"] = np.median(input_df[pred_cols].values, axis=1)

    input_df["pred_std"] = input_df[pred_cols].std(axis=1)
    input_df["pred_max"] = input_df[pred_cols].values.max(axis=1)
    input_df["pred_min"] = input_df[pred_cols].values.min(axis=1)
    input_df["pred_max-min"] = input_df["pred_max"] - input_df["pred_min"]
    input_df["pred_max-median"] = input_df["pred_max"] - input_df["pred_median"]
    input_df["pred_max-mean"] = input_df["pred_max"] - input_df["pred_mean"]
    input_df["pred_median-min"] = input_df["pred_median"] - input_df["pred_min"]
    input_df["pred_mean-min"] = input_df["pred_mean"] - input_df["pred_min"]
    input_df["pred_mean-median"] = input_df["pred_mean"] - input_df["pred_median"]
    input_df["pred_kurt"] = input_df[pred_cols].kurt(axis=1)
    for col_ in pred_cols:
        input_df[f"{col_}_past_1"] = input_df.groupby("breath_id")[f"{col_}"].shift(1)
        input_df[f"{col_}_past_2"] = input_df.groupby("breath_id")[f"{col_}"].shift(2)
        input_df[f"{col_}_past_3"] = input_df.groupby("breath_id")[f"{col_}"].shift(3)
        input_df[f"{col_}_past_4"] = input_df.groupby("breath_id")[f"{col_}"].shift(4)

        input_df[f"{col_}_diff_1"] = input_df[f"{col_}"] - input_df[f"{col_}_past_1"]
        input_df[f"{col_}_diff_2"] = input_df[f"{col_}"] - input_df[f"{col_}_past_2"]
        input_df[f"{col_}_diff_3"] = input_df[f"{col_}"] - input_df[f"{col_}_past_3"]
        input_df[f"{col_}_diff_4"] = input_df[f"{col_}"] - input_df[f"{col_}_past_4"]

    input_df["u_in_past_1"] = input_df.groupby("breath_id")["u_in"].shift(1)
    input_df["u_in_past_2"] = input_df.groupby("breath_id")["u_in"].shift(2)
    input_df["u_in_past_3"] = input_df.groupby("breath_id")["u_in"].shift(3)
    input_df["u_in_past_4"] = input_df.groupby("breath_id")["u_in"].shift(4)

    input_df["u_in_diff_1"] = input_df["u_in"] - input_df["u_in_past_1"]
    input_df["u_in_diff_2"] = input_df["u_in"] - input_df["u_in_past_2"]
    input_df["u_in_diff_3"] = input_df["u_in"] - input_df["u_in_past_3"]
    input_df["u_in_diff_4"] = input_df["u_in"] - input_df["u_in_past_4"]

    input_df["u_in_cumsum"] = input_df.groupby("breath_id")["u_in"].cumsum()

    del input_df['breath_id']
    
    return input_df

In [27]:
train_df = add_oof_features(train_df, dataType = 'train')
test_df = add_oof_features(test_df, dataType = 'test')

In [28]:
display(train_df), display(test_df)

,C,R,R_C,time_step,u_in,pred_0,pred_1,pred_2,pred_3,pred_4,...,pred_9_diff_4,u_in_past_1,u_in_past_2,u_in_past_3,u_in_past_4,u_in_diff_1,u_in_diff_2,u_in_diff_3,u_in_diff_4,u_in_cumsum
0,50,20,5,0.000000,0.083334,6.051358,6.008679,5.839750,5.916682,5.907529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.083334
1,50,20,5,0.033652,18.383041,5.901721,5.824164,5.896797,5.886481,5.899961,...,NaN,0.083334,NaN,NaN,NaN,18.299707,NaN,NaN,NaN,18.466375
2,50,20,5,0.067514,22.509278,8.389902,8.267651,8.516665,8.561444,8.385892,...,NaN,18.383041,0.083334,NaN,NaN,4.126236,22.425944,NaN,NaN,40.975653
3,50,20,5,0.101542,22.808822,12.343451,12.020198,12.398529,12.385787,12.373862,...,NaN,22.509278,18.383041,0.083334,NaN,0.299544,4.425781,22.725488,NaN,63.784476
4,50,20,5,0.135756,25.355850,12.681964,12.505707,12.786497,12.760039,12.763080,...,6.697906,22.808822,22.509278,18.383041,0.083334,2.547028,2.846573,6.972809,25.272516,89.140326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,10,50,6,0.834147,1.869367,29.430931,29.476478,29.435886,29.438047,29.445435,...,1.026603,2.650333,2.438287,3.783043,3.209590,-0.780965,-0.568920,-1.913676,-1.340223,238.890288
2290964,10,50,6,0.867574,2.154414,29.093206,29.113724,29.106075,29.106499,29.109507,...,1.305693,1.869367,2.650333,2.438287,3.783043,0.285047,-0.495918,-0.283873,-1.628629,241.044703
2290965,10,50,6,0.900917,1.304434,29.850384,29.908362,29.861372,29.865599,29.871944,...,0.842991,2.154414,1.869367,2.650333,2.438287,-0.849980,-0.564933,-1.345899,-1.133853,242.349137
2290966,10,50,6,0.934309,1.733830,29.370970,29.400613,29.380976,29.381800,29.382881,...,0.618536,1.304434,2.154414,1.869367,2.650333,0.429396,-0.420585,-0.135538,-0.916503,244.082966


,C,R,R_C,time_step,u_in,pred_0,pred_1,pred_2,pred_3,pred_4,...,pred_9_diff_4,u_in_past_1,u_in_past_2,u_in_past_3,u_in_past_4,u_in_diff_1,u_in_diff_2,u_in_diff_3,u_in_diff_4,u_in_cumsum
0,20,5,1,0.000000,0.000000,6.122457,6.234838,6.228402,6.223941,6.215096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1,20,5,1,0.031904,7.515046,5.897243,5.966046,5.976529,5.962824,5.969975,...,NaN,0.000000,NaN,NaN,NaN,7.515046,NaN,NaN,NaN,7.515046
2,20,5,1,0.063827,14.651675,7.116149,7.154629,7.155302,7.151405,7.149951,...,NaN,7.515046,0.000000,NaN,NaN,7.136630,14.651675,NaN,NaN,22.166721
3,20,5,1,0.095751,21.230610,7.687282,7.767205,7.695719,7.696728,7.700007,...,NaN,14.651675,7.515046,0.000000,NaN,6.578935,13.715564,21.230610,NaN,43.397331
4,20,5,1,0.127644,26.320956,9.229881,9.256029,9.214653,9.217604,9.215394,...,2.991620,21.230610,14.651675,7.515046,0.0,5.090346,11.669281,18.805911,26.320956,69.718287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1527560,10,20,3,0.842145,0.000000,10.105703,10.115388,10.103384,10.102245,10.104448,...,0.026887,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,68.839645
1527561,10,20,3,0.875648,0.000000,10.094866,10.101979,10.089975,10.088987,10.092776,...,-0.014565,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,68.839645
1527562,10,20,3,0.909185,0.121375,9.985191,9.985294,9.983527,9.980724,9.980343,...,-0.142230,0.000000,0.000000,0.000000,0.0,0.121375,0.121375,0.121375,0.121375,68.961019
1527563,10,20,3,0.943148,0.000000,10.079302,10.056884,10.080218,10.079402,10.080560,...,-0.070471,0.121375,0.000000,0.000000,0.0,-0.121375,0.000000,0.000000,0.000000,68.961019


(None, None)

In [29]:
train_category_col = ['R_C']
train_value_col = [i for i in train_df.columns.to_list() if i not in train_category_col]

In [30]:
# norm_features = train_value_col
# train_df = train_df.fillna(0)
# test_df = test_df.fillna(0)
# scaler = RobustScaler()
# scaler.fit(train_df[norm_features])
# train_df[norm_features] = scaler.transform(train_df[norm_features].values)
# test_df[norm_features] = scaler.transform(test_df[norm_features].values)

In [31]:
y = train.loc[train["u_out"] == 0]['pressure'].reset_index(drop=True)

In [32]:
train_df = pd.concat([train_df, train.loc[train["u_out"] == 0, ['id', 'breath_id', 'pressure']].reset_index(drop=True)], axis=1)
test_df = pd.concat([test_df, test.loc[test["u_out"] == 0,['id', 'breath_id']].reset_index(drop=True)], axis=1)

In [33]:
oof_prediction = np.zeros(len(train_df))
test_preds_lst = []
input_dim = len(train_value_col)
train_df['pred'] = 0
train_gby = train_df.groupby('breath_id')['R_C'].agg('first').reset_index()
skf = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
models = []


fold_df = pd.DataFrame()
fold_df["id"] = train["id"]
fold_df["fold"] = -1

for fold, (_, valid_idx) in enumerate(skf.split(train_gby, train_gby['R_C'])):        
    valid_b_ids = train_gby.iloc[valid_idx]['breath_id'].values
    valid_df_idx = train_df[train_df['breath_id'].isin(valid_b_ids)].index.to_list()
    fold_df.loc[valid_df_idx, 'fold'] = fold

for i, fold in enumerate(range(CFG.n_folds)):
    if i not in CFG.folds:
        continue
    print(f'Fold-{fold}')
    
    train_idx = fold_df[fold_df["fold"] != fold].index
    valid_idx = fold_df[fold_df["fold"] == fold].index
    
    trn_df = train_df.loc[fold_df["fold"] != fold, train_value_col].reset_index(drop=True)
    val_df = train_df.loc[fold_df["fold"] == fold, train_value_col].reset_index(drop=True)
    trn_y = train_df.loc[fold_df["fold"] != fold, 'pressure'].reset_index(drop=True)
    val_y = train_df.loc[fold_df["fold"] == fold, 'pressure'].reset_index(drop=True)
    

    lgb_train = lgb.Dataset(trn_df, trn_y)
    lgb_valid = lgb.Dataset(val_df, val_y, reference=lgb_train)

    # LightGBM parameters
    params = {
            'boosting_type': 'gbdt',
             'objective': 'l1',
            'metric': 'l1',
            'seed': CFG.seed,
            'max_bin': 3880, 
            'min_data_in_leaf': 1217, 
            'num_leaves': 36,
            'num_boost_round': 10000,
            'learning_rate': 0.1,
            'lambda_l1': 0.1,
            'lambda_l2': 0.1,
            'n_jobs': -1
    }


    model = lgb.train(params, 
                      train_set=lgb_train, 
                      valid_sets=[lgb_train, lgb_valid], early_stopping_rounds=200, verbose_eval=100)
    
    models.append(model)
    
    oof_prediction[valid_idx] = model.predict(val_df[train_value_col])
    test_pred = model.predict(test_df[train_value_col])
    test_preds_lst.append(test_pred)
    score = np.abs(val_y.values - oof_prediction[valid_idx]).mean()
    print(f'fold = {fold}, score = {score}')

Fold-0
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.370007 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 438448
[LightGBM] [Info] Number of data points in the train set: 2061893, number of used features: 115
[LightGBM] [Info] Start training from score 15.820396
Training until validation scores don't improve for 200 rounds
[100]	training's l1: 0.12971	valid_1's l1: 0.128368
[200]	training's l1: 0.123324	valid_1's l1: 0.122029
[300]	training's l1: 0.121308	valid_1's l1: 0.120101
[400]	training's l1: 0.120124	valid_1's l1: 0.118985
[500]	training's l1: 0.119277	valid_1's l1: 0.118219
[600]	training's l1: 0.118731	valid_1's l1: 0.117785
[700]	training's l1: 0.118391	valid_1's l1: 0.117539
[800]	training's l1: 0.118179	valid_1's l1: 0.117407
[900]	training's l1: 0.117949	valid_1's l1: 0.117255
[1000]	training's l1: 0.117686	valid_1's l1: 0.117108
[1100]	training's l1: 0.117446	valid_1's l1: 0.116

In [34]:
CV = np.abs(y.values - oof_prediction).mean()
CV

0.11764851501990573

In [35]:
np.save(OUTPUT_DIR / f"stacking2_oof_{CFG.exp_num}", oof_prediction)

In [36]:
unique_pressures = train["pressure"].unique()
sorted_pressures = np.sort(unique_pressures)
total_pressures_len = len(sorted_pressures)

def find_nearest(prediction):
    insert_idx = np.searchsorted(sorted_pressures, prediction)
    if insert_idx == total_pressures_len:
        # If the predicted value is bigger than the highest pressure in the train dataset,
        # return the max value.
        return sorted_pressures[-1]
    elif insert_idx == 0:
        # Same control but for the lower bound.
        return sorted_pressures[0]
    lower_val = sorted_pressures[insert_idx - 1]
    upper_val = sorted_pressures[insert_idx]
    return lower_val if abs(lower_val - prediction) < abs(upper_val - prediction) else upper_val

In [37]:
oof = pd.DataFrame({'pred': oof_prediction})
oof_pp = oof['pred'].map(lambda x: unique_pressures[np.abs(unique_pressures-x).argmin()])
score = np.abs(y.values - oof_pp).mean()
print(score)

0.1161824389827335


In [38]:
sub_df = pd.read_csv(DATA_DIR / "sample_submission.csv")

sub_df.loc[test['u_out']==0, 'pressure'] = np.stack(test_preds_lst).mean(0)
sub_df.to_csv(OUTPUT_DIR / f"stacking2_submission_mean_{CFG.exp_num}.csv", index=None)

sub_df.loc[test['u_out']==0, 'pressure'] = np.median(np.stack(test_preds_lst), axis=0)
sub_df.to_csv(OUTPUT_DIR / f"stacking2_submission_median_{CFG.exp_num}.csv", index=None)

# Post Processing: https://www.kaggle.com/snnclsr/a-dummy-approach-to-improve-your-score-postprocess


sub_df = pd.read_csv(OUTPUT_DIR / f"stacking2_submission_mean_{CFG.exp_num}.csv")
sub_df.loc[test['u_out']==0, 'pressure'] = sub_df["pressure"].apply(find_nearest)
sub_df.to_csv(OUTPUT_DIR / f"stacking2_submission_mean_pp_{CFG.exp_num}.csv", index=None)

sub_df = pd.read_csv(OUTPUT_DIR / f"stacking2_submission_median_{CFG.exp_num}.csv")
sub_df.loc[test['u_out']==0, 'pressure'] = sub_df["pressure"].apply(find_nearest)
sub_df.to_csv(OUTPUT_DIR / f"stacking2_submission_median_pp_{CFG.exp_num}.csv", index=None)